In [86]:
import time
import random
import requests
import selenium
from string import ascii_lowercase
from selenium.webdriver import Firefox, Chrome
from bs4 import BeautifulSoup
import pandas as pd
import string
import numpy as np
from selenium.webdriver.common.keys import Keys

In [87]:
browser = Chrome()

In [88]:
url='https://www.allrecipes.com/recipe/141833/thai-green-curry-chicken/'
browser.get(url)

In [84]:
time.sleep(2)

In [9]:
sel = 'ul'
ing_lists = browser.find_elements_by_css_selector(sel)

In [11]:
sel = 'ul#lst_ingredients_1'
ing_list_1 = browser.find_element_by_css_selector(sel)

In [12]:
ing_list_1.text.split('\n')

['1 pound skinless, boneless chicken breast halves - cut into 1 inch cubes',
 '1 tablespoon dark soy sauce',
 '1 tablespoon all-purpose flour',
 'Great Value All Purpose Flour, 32 oz',
 'In Stores Only',
 'SEE EVERYDAY LOW PRICE',
 'ADVERTISEMENT',
 '2 tablespoons cooking oil',
 '2 tablespoons green curry paste',
 '2 green onions with tops, chopped',
 '3 cloves garlic, peeled and chopped']

In [10]:
ing_lists.text.split('\n')

AttributeError: 'list' object has no attribute 'text'

In [13]:
sel = 'ul#lst_ingredients_2'
ing_list_2 = browser.find_element_by_css_selector(sel)

In [14]:
ing_list_2.text.split('\n')

['']

In [15]:
ing_lists = ing_list_1.text.split('\n') + ing_list_2.text.split('\n')

In [16]:
ingredients = []
for item in ing_lists:
    if item[0] in string.digits:
        ingredients.append(item)

IndexError: string index out of range

In [17]:
ingredients

[]

In [49]:
quantities = []
units = []
ingredient = []
for item in ingredients:
    quantities.append(item.split()[0])
    units.append(item.split()[1])
    ingredient.append(' '.join(item.split()[2:]))

In [1]:
units

NameError: name 'units' is not defined

In [90]:
def get_ingredients():
    all_items = []
    count = 1
    while True:
        try:
            sel = 'ul#lst_ingredients_{0}'.format(count)
            ing_list = browser.find_element_by_css_selector(sel)
            all_items += ing_list.text.split('\n')
            count += 1
        except:
            break
    ingredients = []
    for item in all_items:
        if item[0] in string.digits:
            ingredients.append(item)
    return ingredients

In [104]:
ingredients = get_ingredients()
ingredients

['1 pound skinless, boneless chicken breast halves - cut into 1 inch cubes',
 '1 tablespoon dark soy sauce',
 '1 tablespoon all-purpose flour',
 '2 tablespoons cooking oil',
 '2 tablespoons green curry paste',
 '2 green onions with tops, chopped',
 '3 cloves garlic, peeled and chopped',
 '1 teaspoon fresh ginger, peeled and finely chopped',
 '2 cups coconut milk',
 '1 tablespoon fish sauce',
 '1 tablespoon dark soy sauce',
 '2 tablespoons white sugar',
 '1/2 cup cilantro leaves, for garnish']

In [110]:
def parse_ingredients(ingredients, units):
    '''
    Parses a list of ingredients into a list of dictionaries with the following format: 
        {'ingredient': (str),
         'quantity': (float),
         'units': (str),
         'preparation': (str)}
    Also takes argument 'units', a list of accepted units (e.g., ['cups', 'tablespoon']).
    If an ingredident does not specify a unit in this list, the label 'each' will be applied.
    '''
    ing_list = []
    for item in ingredients:
        item_dict = {}
        
        # Determine quantity
        quantity = item.split()[0]
        try:
            item_dict['quantity'] = float(quantity)
        except:
            numer, denom = quantity.split('/')
            item_dict['quantity'] = float(numer) / float(denom)
        
        # Determine units
        if item.split()[2] not in ['can', 'cans']:
            if item.split()[1] in units:
                item_dict['units'] = item.split()[1]
                remainder = ' '.join(item.split()[2:])
            else:
                item_dict['units'] = 'each'
                remainder = ' '.join(item.split()[1:])
        else:
            item_dict['units'] = item.split()[1:3]
            remainder = ' '.join(item.split()[3:])
            
        # Split remaining text between ingredient and preparation
        if len(remainder.split(' - ')) != 1:
            item_dict['ingredient'] = remainder.split(' - ')[0]
            item_dict['preparation'] = remainder.split(' - ')[1]
        else:
            item_dict['ingredient'] = remainder.split(', ')[0]
            if len(remainder.split(', ')) != 1:
                item_dict['preparation'] = remainder.split(', ')[1]
            else:
                item_dict['preparation'] = None
                
        # Add item dictionary to list
        ing_list.append(item_dict)
    
    return ing_list

In [105]:
units = ['pound', 'pounds', 'cup', 'cups', 'tablespoon', 'tablespoons', 'teaspoon', 'teaspoons',
         'clove', 'cloves']

In [111]:
parse_ingredients(ingredients, units)

[{'quantity': 1.0,
  'units': 'pound',
  'ingredient': 'skinless, boneless chicken breast halves',
  'preparation': 'cut into 1 inch cubes'},
 {'quantity': 1.0,
  'units': 'tablespoon',
  'ingredient': 'dark soy sauce',
  'preparation': None},
 {'quantity': 1.0,
  'units': 'tablespoon',
  'ingredient': 'all-purpose flour',
  'preparation': None},
 {'quantity': 2.0,
  'units': 'tablespoons',
  'ingredient': 'cooking oil',
  'preparation': None},
 {'quantity': 2.0,
  'units': 'tablespoons',
  'ingredient': 'green curry paste',
  'preparation': None},
 {'quantity': 2.0,
  'units': 'each',
  'ingredient': 'green onions with tops',
  'preparation': 'chopped'},
 {'quantity': 3.0,
  'units': 'cloves',
  'ingredient': 'garlic',
  'preparation': 'peeled and chopped'},
 {'quantity': 1.0,
  'units': 'teaspoon',
  'ingredient': 'fresh ginger',
  'preparation': 'peeled and finely chopped'},
 {'quantity': 2.0,
  'units': 'cups',
  'ingredient': 'coconut milk',
  'preparation': None},
 {'quantity': 1

In [95]:
def get_name():
    sel = 'h1#recipe-main-content'
    name = browser.find_element_by_css_selector(sel)
    return name.text

In [98]:
get_name()

'Thai Green Curry Chicken'

In [93]:
def get_rating():
    sel = 'div.rating-stars'
    rating = browser.find_element_by_css_selector(sel)
    return float(rating.get_attribute('data-ratingstars'))

In [94]:
get_rating()

4.53484010696411

In [96]:
def get_categories():
    sel = 'ol.breadcrumbs li'
    categories = browser.find_elements_by_css_selector(sel)
    return [category.text for category in categories]

In [97]:
get_categories()

['Home', 'Recipes', 'World Cuisine', 'Asian', 'Thai']

In [100]:
a = '1/2'

In [101]:
x, y = a.split('/')

In [102]:
x

'1'